In [ ]:
!pip install swifter

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import log
import seaborn as sns
import swifter

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error as msle
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

In [ ]:
# !apt-get install p7zip
# !p7zip -d -f -k /kaggle/input/mercari-price3-suggestion-challenge/train.tsv.7z
# !p7zip -d -f -k /kaggle/input/mercari-price3-suggestion-challenge/test.tsv.7z
# !p7zip -d -f -k /kaggle/input/mercari-price3-suggestion-challenge/test_stg2.tsv.7z

In [ ]:
train = pd.read_csv("../input/mercari/train.tsv", sep='\t').set_index("train_id")
# test = pd.read_csv("../input/mercari/test.tsv", sep='\t').set_index("test_id")
tstg2 = pd.read_csv("../input/mercari/test_stg2.tsv", sep='\t').set_index("test_id")



# train = train.reset_index()
# test = test.reset_index()
# ex_sub = pd.read_csv("sample_submission.csv")

In [ ]:
train.head()

In [ ]:
# test.head()

In [ ]:
tstg2.head()

In [ ]:
# print(train.shape, test.shape, tstg2.shape)
print(train.shape, tstg2.shape)

ターゲットの確認

In [ ]:
train["price"].describe()

0$のレコードが存在する　売れたかどうかということも使うか(→売れたかどうかを予測したのち、売れない→0　売れた→さらに予測　分類のあとに回帰)

priceなので予測値が負の値を取らない→負だったら0に置き換えたり処理が必要(特に今回の評価指標はrmsleなので予測値が-1以下の値をとると評価できない)

売れなかった(priceが0$)だったレコードがいくつあるか確認

約150万個中の874件→0.05%なので分類→回帰はリスキーか

In [ ]:
train[train["price"] == 0]["price"].count()

売れなかったレコードの変数を見てみる

In [ ]:
train[train["price"] == 0].head()

ターゲットのヒストグラムの確認(0を抜いたヒストグラムも),95%点以下に絞ったヒストグラム

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18,6))
x1, bins1, patches1 = axes[0].hist(train["price"], bins=10)
# x2, bins2, patches2 = axes[1].hist(train[train["price"] != 0]["price"], bins=10) 
x2, bins2, patches2 = axes[1].hist(train[train["price"]<=train["price"].quantile(q=0.95)]["price"], bins=10)
x3, bins3, patches3 = axes[2].hist(train[(train["price"]<=train["price"].quantile(q=0.95)) & (train["price"] != 0)]["price"], bins=10)

axes[0].set_title("standard")
axes[1].set_title("<=95%")
axes[2].set_title("<=95% & !=0")

print(x1, "\n", bins1)
print(x2, "\n", bins2)
print(x3, "\n", bins3)

95%点以下で0以外の値を取るターゲットに関して対数変換

In [ ]:
ln_target = (train[(train["price"] <= train["price"].quantile(q=0.95)) & (train["price"] != 0)]["price"]).apply(log)
# ln_target = (train[train["price"] != 0]["price"]).apply(log)
ln_target.head()

対数変換データと0データを結合

In [ ]:
tmp_target = train[(train["price"] <= train["price"].quantile(q=0.95)) & (train["price"] == 0)]["price"].append(ln_target)

In [ ]:
# fig, ax = plt.subplots(figsize=(16,6))
# x, bins, patches = ax.hist(tmp_target, bins=10)
# print(x, "\n", bins)

fig, axes = plt.subplots(2,1,figsize=(16,12))
axes[0].hist(train["price"], bins=10)
axes[1].hist(tmp_target, bins=10)


尖度と歪度を確認

In [ ]:
print(train["price"].skew(), train["price"].kurt())
print(tmp_target.skew(), tmp_target.kurt())

target 0と95%点超は学習データから除外し、対数変換　予測後に指数関数で戻して、予測値が負の値をとった場合これを0に変換する?

In [ ]:
train[(train["price"]>train["price"].quantile(q=0.95)) | (train["price"] == 0)]["price"].count()

148万件とすると5%くらいが除外対象となる

数値変数の値について確認

In [ ]:
print(train["item_condition_id"].groupby(train["item_condition_id"]).count())
print('------------')
print(train["shipping"].groupby(train["shipping"]).count())
print('------------')
# print(test["item_condition_id"].groupby(test["item_condition_id"]).count())
# print('------------')
# print(test["shipping"].groupby(test["shipping"]).count())
# print('------------')
print(tstg2["item_condition_id"].groupby(tstg2["item_condition_id"]).count())
print('------------')
print(tstg2["shipping"].groupby(tstg2["shipping"]).count())


In [ ]:
# train1 = pd.concat([tmp_train, pd.get_dummies(tmp_train["item_condition_id"], prefix='condition_')], axis=1).drop("item_condition_id", axis=1)
# test1 = pd.concat([tmp_test, pd.get_dummies(tmp_test["item_condition_id"], prefix='condition_')], axis=1).drop("item_condition_id", axis=1)
train1 = train
# test1 = test
tstg21 = tstg2
train1.head()

次にcategory_nameについて/区切りをバラすといくつ値を持っているのか確認

In [ ]:
# llist = []
# for i in range(len(train)):
#     if type(train["category_name"][i]) is str:
#         llist.append(len(train["category_name"][i].split('/')))
# print(min(llist), max(llist))

In [ ]:
# llist2 = []
# for i in range(len(test)):
#     if type(test["category_name"][i]) is str:
#         llist2.append(len(test["category_name"][i].split('/')))
# print(min(llist2), max(llist2))

In [ ]:
# llist3 = []
# for i in range(len(tstg2)):
#     if type(tstg2["category_name"][i]) is str:
#         llist3.append(len(tstg2["category_name"][i].split('/')))
# print(min(llist3), max(llist3))

最大5つ値を持っているが基本的に最小の3つを候補としてみていく

In [ ]:
tr_cat = train["category_name"].str.split('/', expand=True).add_prefix('cat_')
# te_cat = test["category_name"].str.split('/', expand=True).add_prefix('cat_')
ts_cat = tstg2["category_name"].str.split('/', expand=True).add_prefix('cat_')

In [ ]:
tr_cat.head()
# te_cat.head()
# ts_cat.head()

各カテゴリ内の値に関する出現数を確認

cat_1は最小値でも31回　なのに対してcat_2は1度しか出現していない値が少なくとも1つある

そのため、cat_2に関しては説明変数として採用する際は下限を決めて絞る必要があるかもしれない

In [ ]:
print(tr_cat["cat_1"].groupby(tr_cat["cat_1"]).count().sort_values().describe())
print("-------------------------------")
print(tr_cat["cat_2"].groupby(tr_cat["cat_2"]).count().sort_values().describe())

cat_0,cat_1についてどんな値が入っているか重複をはじいて上から20件を確認

In [ ]:
tr_cat[["cat_0", "cat_1"]].sort_values(["cat_0", "cat_1"]).drop_duplicates().head(20)

カテゴリごとの平均priceとその分散を確認

In [ ]:
# conf_cat = pd.concat([tr_cat["cat_0"], train["price"]], axis=1)
conf_cat = pd.concat([tr_cat["cat_0"], tmp_target], axis=1).groupby(tr_cat["cat_0"]).mean()
display(conf_cat.sort_values(by="price"))
print(conf_cat.sort_values(by="price").var())

In [ ]:
conf_cat = pd.concat([tr_cat["cat_1"], tmp_target], axis=1).groupby(tr_cat["cat_1"]).mean()
display(conf_cat.sort_values(by="price").describe())
print(conf_cat.sort_values(by="price").var())

In [ ]:
conf_cat = pd.concat([tr_cat["cat_2"], tmp_target], axis=1).groupby(tr_cat["cat_2"]).mean()
display(conf_cat.sort_values(by="price").describe())
print(conf_cat.sort_values(by="price").var())

cat_0に関してそれぞれpriceのヒストグラムを描画

In [ ]:
cat0_name = tr_cat["cat_0"].drop_duplicates().values

In [ ]:
# fig, axes = plt.subplots(11,figsize=(16,55))
# for i,name in enumerate(cat0_name):
#     axes[i].hist(conf_cat[conf_cat["cat_0"] == name]["price"], bins=20)
#     axes[i].set_title(name)
    
# ax.hist(conf_cat[conf_cat["cat_0"] == "Men"]["price"])

cat_0に関して、学習とテストのどちらか一方にしかないようなカテゴリがないか確認

In [ ]:
print(np.sort(tr_cat["cat_0"].fillna("NaN").drop_duplicates().values))
# print(np.sort(te_cat["cat_0"].fillna("NaN").drop_duplicates().values))
print(np.sort(ts_cat["cat_0"].fillna("NaN").drop_duplicates().values))


# print(te_cat["cat_0"].sort_values("cat_0").drop_duplicates())

抽出したカテゴリ変数を結合

In [ ]:
train1.head()

In [ ]:
train2 = pd.concat([train1.drop("category_name", axis=1), pd.get_dummies(tr_cat["cat_0"],prefix="cat0")], axis=1)
# test2 = pd.concat([test1.drop("category_name", axis=1), pd.get_dummies(te_cat["cat_0"],prefix="cat0")], axis=1)
tstg22 = pd.concat([tstg21.drop("category_name", axis=1), pd.get_dummies(ts_cat["cat_0"],prefix="cat0")], axis=1)

cat_1追加　ここから

In [ ]:
print(tr_cat["cat_1"].groupby(tr_cat["cat_1"]).count().count())
# print(te_cat["cat_1"].groupby(te_cat["cat_1"]).count().count())
print(ts_cat["cat_1"].groupby(ts_cat["cat_1"]).count().count())

In [ ]:
tr_a = tr_cat["cat_1"].groupby(tr_cat["cat_1"]).count().index.drop_duplicates()
# te_a = te_cat["cat_1"].groupby(te_cat["cat_1"]).count().index.drop_duplicates()
ts_a = ts_cat["cat_1"].groupby(ts_cat["cat_1"]).count().index.drop_duplicates()

cat_1に関して、学習とテストのどちらか一方にしかないようなカテゴリがないか確認

In [ ]:
for a in tr_a:
    if a not in ts_a:
        print(a)
print("end")
for a in ts_a:
    if a not in tr_a:
        print(a)
print("end")

In [ ]:
train2 = pd.concat([train2, pd.get_dummies(tr_cat["cat_1"], prefix="cat1")], axis=1)
# test2 = pd.concat([test2, pd.get_dummies(te_cat["cat_1"], prefix="cat1")], axis=1)
tstg22 = pd.concat([tstg22, pd.get_dummies(ts_cat["cat_1"], prefix="cat1")], axis=1)

In [ ]:
tmp = train2.isnull().sum()
print(tmp[tmp>0])

In [ ]:
# print(train2.shape, test2.shape, tstg22.shape)
print(train2.shape, tstg22.shape)

cat_1追加　ここまで

cat_2追加　ここから

In [ ]:
print(tr_cat["cat_2"].groupby(tr_cat["cat_2"]).count().count())
# print(te_cat["cat_2"].groupby(te_cat["cat_2"]).count().count())
print(ts_cat["cat_2"].groupby(ts_cat["cat_2"]).count().count())

In [ ]:
tr_a = tr_cat["cat_2"].groupby(tr_cat["cat_2"]).count().index.drop_duplicates()
# te_a = te_cat["cat_2"].groupby(te_cat["cat_2"]).count().index.drop_duplicates()
ts_a = ts_cat["cat_2"].groupby(ts_cat["cat_2"]).count().index.drop_duplicates()

一方にしかないような値の確認

In [ ]:
for a in tr_a:
    if a not in ts_a:
        print(a)
print("end")
for a in ts_a:
    if a not in tr_a:
        print(a)
print("end")

cat_2追加　ここまで

追加するか迷う　次元削減するのであれば追加してもいいかも→テストデータにしかない方の値についてどのくらい影響力があるかわかない

In [ ]:
train2.head()

ブランド名について

In [ ]:
print(train["brand_name"].groupby(train["brand_name"]).count().count())
# print(test["brand_name"].groupby(test["brand_name"]).count().count())
print(tstg2["brand_name"].groupby(tstg2["brand_name"]).count().count())

In [ ]:
tr_a = train["brand_name"].groupby(train["brand_name"]).count().index.drop_duplicates()
# te_a = test["brand_name"].groupby(test["brand_name"]).count().index.drop_duplicates()
ts_a = tstg2["brand_name"].groupby(tstg2["brand_name"]).count().index.drop_duplicates()

In [ ]:
for a in tr_a:
    if a not in ts_a:
        print(a)
print("end")
for a in ts_a:
    if a not in tr_a:
        print(a)
print("end")

ブランド名ごとにプライスの平均値を確認し、クラスタリング

ブランドごとにデータ数に開きがあるため、平均値(などの代表値)に信頼性を持たせたい

In [ ]:
brand_price = train["price"].groupby(train["brand_name"]).mean().sort_values().fillna("no brand").reset_index()

In [ ]:
display(brand_price.head())

In [ ]:
brand_price.describe()

In [ ]:
brand_price[brand_price["price"]<=brand_price["price"].quantile(q=0.95)].hist(bins=20)

In [ ]:
brand_price.quantile(q=0.5)[0]

各ブランドの平均値を3つのランクに分割し、ブランド名の代わりに私用する(ただし、ブランド名の半分近くはnan)

In [ ]:
def brand_rank(price):
    if price <= brand_price.quantile(q=0.25)[0]:
        return "L"
    elif brand_price.quantile(q=0.25)[0] < price <= brand_price.quantile(q=0.75)[0]:
        return "M"
    else:
        return "H"

In [ ]:
brand_price["brand_rank"] = brand_price["price"].swifter.apply(brand_rank)
brand_rank = brand_price.drop("price", axis=1)

In [ ]:
brand_rank.head()

In [ ]:
rank_L = brand_rank[brand_rank["brand_rank"] == "L"]["brand_name"].values
rank_M = brand_rank[brand_rank["brand_rank"] == "M"]["brand_name"].values
rank_H = brand_rank[brand_rank["brand_rank"] == "H"]["brand_name"].values

In [ ]:
# train3 = pd.merge(train2,brand_rank,how="left")
# train3.head()

In [ ]:
len(brand_rank)
brand_rank.shape

In [ ]:
def brand_rank_app(brand_name):
#     for i in range(len(brand_rank)):
#         if brand_name == brand_rank.iloc[i,0]:
#             return brand_rank.iloc[i,1]
    for lmh, rank in zip(["L","M","H"],[rank_L,rank_M,rank_H]):
        if brand_name in rank:
            return lmh
    else:
        return "other"

In [ ]:
tmp = train2["brand_name"].swifter.apply(brand_rank_app)
tmp_tr = pd.DataFrame(tmp.values, columns={"brand_rank"})
# tmp = test2["brand_name"].swifter.apply(brand_rank_app)
# tmp_te = pd.DataFrame(tmp.values, columns={"brand_rank"})
tmp = tstg22["brand_name"].swifter.apply(brand_rank_app)
tmp_ts = pd.DataFrame(tmp.values, columns={"brand_rank"})

In [ ]:
rank_tr_dum = pd.get_dummies(tmp_tr, prefix="rank").drop("rank_other", axis=1)
# rank_te_dum = pd.get_dummies(tmp_te, prefix="rank").drop("rank_other", axis=1)
rank_ts_dum = pd.get_dummies(tmp_ts, prefix="rank").drop("rank_other", axis=1)

In [ ]:
# print(rank_tr_dum.shape, rank_te_dum.shape, rank_ts_dum.shape)
print(rank_tr_dum.shape, rank_ts_dum.shape)

In [ ]:
train2.columns

In [ ]:
# train3 = pd.merge(train2, tmp_tr, how="outer")
train3 = pd.concat([train2, rank_tr_dum], axis=1).drop("brand_name", axis=1)
# test3 = pd.concat([test2, rank_te_dum], axis=1).drop("brand_name", axis=1)
tstg23 = pd.concat([tstg22, rank_ts_dum], axis=1).drop("brand_name", axis=1)
# display(train3.head(), test3.head(), tstg23.head())
display(train3.head(), tstg23.head())

In [ ]:
# print(train3.columns, test3.columns, tstg23.columns)
print(train3.columns, tstg23.columns)

### 説明変数として使うもの

コンディション、送料をどちらが持つか、商品カテゴリの上二層、ブランドのランク

変数選択　ターゲットとの相関関係を見てreject → PCA

In [ ]:
train3.corr()["price"].abs().sort_values()

In [ ]:
X, y = train3.drop(["name", "price", "item_description"], axis=1), train3["price"]
# X, y = train3.drop(["name", "price", "item_description", "item_condition_id", "cat0_Vintage & Collectibles", "cat0_Sports & Outdoors"], axis=1), train3["price"]
# X, y = train3[["rank_H", "rank_M"]], train3["price"]
# testX = test3.drop(["name", "item_description"], axis=1)
tstg2X = tstg23.drop(["name", "item_description"], axis=1)
display(X.head())

In [ ]:
# print(X.shape, y.shape, testX.shape, tstg2X.shape)
print(X.shape, y.shape, tstg2X.shape)

In [ ]:
# print(X.isnull().sum(), testX.isnull().sum())

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(random_state=42)
pca.fit(X)
feature = pd.DataFrame(pca.transform(X)).add_prefix('pca_')
feature.head()

In [ ]:
a = pd.DataFrame(pca.explained_variance_, index=["PC{}".format(x + 1) for x in range(len(X.columns))], columns=["固有値"])
a["寄与率"] = a["固有値"] / a["固有値"].sum()
a["累積寄与率"] = ""
for i in range(len(a["寄与率"])):
    if i != 0:
        a.at[a.index[i],"累積寄与率"] = a.at[a.index[i], "寄与率"] + a.at[a.index[i-1], "累積寄与率"]
    elif i == 0:
        a.at[a.index[0], "累積寄与率"] = a.at[a.index[0], "寄与率"]
display(a.head(40))
plt.subplots(facecolor='w', figsize=(8,5)) 
plt.grid()
plt.plot(a.index, a["累積寄与率"])
plt.bar(a.index, a["寄与率"], alpha=0.4)
plt.show()

In [ ]:
tmp = pd.DataFrame((-1)*pca.components_, columns=X.columns[:],
                   index=["PC{}".format(x + 1) for x in range(len(X.columns))]).T

In [ ]:
X_pca = feature.iloc[:,0:40]
X_pca.head()

In [ ]:
print(X.shape, X_pca.shape)

In [ ]:
featuret = pd.DataFrame(pca.transform(tstg2X)).add_prefix('pca_')
tstg2X = featuret.iloc[:,0:40]
tstg2X.head()

In [ ]:
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_pca, y, test_size=0.3, random_state=42)
print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)

In [ ]:
# from sklearn.neighbors import KNeighborsRegressor

In [ ]:
# knn = KNeighborsRegressor().fit(X_train, y_train)
# knn_val_pred = knn.predict(X_valid)
# knn_val_pred[knn_val_pred<0] = 0 
# knn_val_rmsle = round(np.sqrt(msle(knn_val_pred, y_valid)), 6)
# print(knn_val_rmsle)

ハイパーパラメータを設定しないシンプルなモデルを作成し、精度を確認

In [ ]:
# lr = LinearRegression().fit(X_train, y_train)
# lr_val_pred = lr.predict(X_valid)
# lr_val_pred[lr_val_pred<0] = 0
# lr_val_rmsle = round(np.sqrt(msle(lr_val_pred, y_valid)), 6)
# print(lr_val_rmsle)

In [ ]:
# origin lr:0.707541 pca lr:0.717154

In [ ]:
# rf = RandomForestRegressor(n_jobs=-1, random_state=42).fit(X_train, y_train)
# gb = GradientBoostingRegressor(random_state=42).fit(X_train, y_train)
# svr = SVR().fit(X_train, y_train)

In [ ]:
# lr_val_pred = lr.predict(X_valid)
# rf_val_pred = rf.predict(X_valid)
# gb_val_pred = gb.predict(X_valid)
# svr_val_pred = svr.predict(X_valid)

# lr_val_rmsle = round(np.sqrt(msle(lr_val_pred, y_valid)), 6)
# rf_val_rmsle = round(np.sqrt(msle(rf_val_pred, y_valid)), 6)
# gb_val_rmsle = round(np.sqrt(msle(gb_val_pred, y_valid)), 6)
# svr_val_rmsle = np.sqrt(msle(svr_val_pred, y_valid))

# print(rf_val_rmsle, gb_val_rmsle)

In [ ]:
# display(X_train.shape,testX.shape)

In [ ]:
# tstg2X.isnull().sum()

In [ ]:
# rf_test = pd.concat([pd.DataFrame(test.index), pd.DataFrame(rf.predict(testX))], axis=1)
# rf_tstg2 = pd.concat([pd.DataFrame(tstg2.index), pd.DataFrame(rf.predict(tstg2X))], axis=1)

# display(rf_tstg2)

In [ ]:
# rf_test.to_csv("sub.csv", index=False)
# rf_tstg2.to_csv("submission.csv", index=False)

In [ ]:
# 1:cat_0まで使用
# 0.721973430594621 0.7054398273035809 0.7079044651718815 すべて使用
# 0.723318 0.715236 0.715431　item_condition_idをreject
# 0.723071 0.715245 0.715312 cat0_下から2つをreject
# 0.744507 0.744617 0.744507 rank_H,rank_Mのみを使用

In [ ]:
# 2:cat_1まで使用
# linear rf gb
# 0.707517 0.655803 0.684233　すべて使用
#          0.655752 0.675928 推定器を250個に　
# nn 0.668589
# rf:gb 37 55 75 アンサンブル
# 0.666783 0.662114 0.658602

In [ ]:
rf1 = RandomForestRegressor(n_estimators=250,  n_jobs=-1, random_state=42).fit(X_train, y_train)

In [ ]:
rf1_val_pred = rf1.predict(X_valid)
rf1_val_rmsle = round(np.sqrt(msle(rf1_val_pred, y_valid)), 6)
print(rf1_val_rmsle)

In [ ]:
# origin rf1:0.655752 pca rf1:0.655758

In [ ]:
# gb1 = GradientBoostingRegressor(n_estimators=200, random_state=42).fit(X_train, y_train)

In [ ]:
# gb1_val_pred = gb1.predict(X_valid)
# gb1_val_rmsle = round(np.sqrt(msle(gb1_val_pred, y_valid)), 6)
# gb1_val_rmsle

In [ ]:
# pca gb1:0.664465

In [ ]:
# def ensemble_rmsle(mod1, mod2, mod1_rate):
#     tmp_pred = mod1.predict(X_valid) * mod1_rate + mod2.predict(X_valid) * (1-mod1_rate)
#     tmp_pred[tmp_pred<0] = 0
#     return round(np.sqrt(msle(tmp_pred, y_valid)), 6)

In [ ]:
# rfgb_37_rmsle = ensemble_rmsle(rf1, gb1, 0.3)
# rfgb_55_rmsle = ensemble_rmsle(rf1, gb1, 0.5)
# rfgb_73_rmsle = ensemble_rmsle(rf1, gb1, 0.7)

# print(rfgb_37_rmsle, rfgb_55_rmsle, rfgb_73_rmsle)

In [ ]:
# 0.660329 0.658286 0.656806

In [ ]:
# rfgb_37_pred = rf1.predict(tstg2X) * 0.3 + gb1.predict(tstg2X) * 0.7
# rfgb_55_pred = rf1.predict(tstg2X) * 0.5 + gb1.predict(tstg2X) * 0.5
# rfgb_73_pred = rf1.predict(tstg2X) * 0.7 + gb1.predict(tstg2X) * 0.3

In [ ]:
# rfgb_37_tstg2 = pd.concat([pd.DataFrame(tstg2.index), pd.DataFrame(rfgb_37_pred)], axis=1)
# rfgb_55_tstg2 = pd.concat([pd.DataFrame(tstg2.index), pd.DataFrame(rfgb_55_pred)], axis=1)
# rfgb_73_tstg2 = pd.concat([pd.DataFrame(tstg2.index), pd.DataFrame(rfgb_73_pred)], axis=1)
rf_tstg2 = pd.concat([pd.DataFrame(tstg2.index), pd.DataFrame(rf1.predict(tstg2X))], axis=1)

In [ ]:
# rfgb_37_tstg2.columns = ["test_id", "price"]
# rfgb_55_tstg2.columns = ["test_id", "price"]
# rfgb_73_tstg2.columns = ["test_id", "price"]
rf_tstg2.columns = ["test_id", "price"]

In [ ]:
# rfgb_37_tstg2.to_csv("submission37.csv", index=False)
# rfgb_55_tstg2.to_csv("submission55.csv", index=False)
# rfgb_73_tstg2.to_csv("submission73.csv", index=False)
rf_tstg2.to_csv("submission.csv", index=False)

In [ ]:
# rf_tstg2.head()

In [ ]:
# nn = MLPRegressor(random_state=42).fit(X_train, y_train)

In [ ]:
# nn_val_pred = nn.predict(X_valid)
# nn_val_pred[nn_val_pred<0] = 0
# nn_rmsle = round(np.sqrt(msle(nn_val_pred, y_valid)), 6)
# print(nn_rmsle)

In [ ]:
# rfgb_37_rmsle = ensemble_rmsle(rf1, nn, 0.3)
# rfgb_55_rmsle = ensemble_rmsle(rf1, nn, 0.5)
# rfgb_73_rmsle = ensemble_rmsle(rf1, nn, 0.7)

# print(rfgb_37_rmsle, rfgb_55_rmsle, rfgb_73_rmsle)

In [ ]:
# 0.658866 0.657435 0.656376

In [ ]:
# nn_pred = pd.concat([pd.DataFrame(tstg2.index), nn.predict(tstg2X)], axis=1)
# nn_pred.columns = ["test_id", "price"]
# nn_pred.to_csv("submission.csv", index=False)

アイテムの説明について → 何か特定のキーワードに目をつけて精度が良くなるかどうか見てみる